In [2]:
import pandas as pd

# Clean investing.com data
# https://ca.investing.com/indices/us-spx-500-historical-data 

def clean_investingcom_df(df, name, country_code):
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df = df.reset_index(drop=True).iloc[::-1]

    df['Change %'] = df['Change %'].str.replace('%', '').astype(float)
    volatility = df.groupby(by='year')['Change %'].std() * (df.groupby(by='year').size())**0.5

    df = df.groupby(by='year').first().reset_index()

    df['volatility'] = volatility.values
    df['YoY Change'] = df['Price'].pct_change()*100
    df['YoY Change'] = df['YoY Change'].shift(-1)

    df['code'] = country_code
    df['year'] = pd.to_datetime(df['Date']).dt.year
    df = df[['year', 'YoY Change', 'volatility', 'code']]
    df.dropna(inplace=True)
    df.to_csv(f'./Datasets/{name}-Clean.csv', index=False)

In [25]:
dataset_list = ['S&P500-USA', 'Shanghai-CHN', 'FTSE_JSE-ZAF', 'MASI-MAR', 'NASI-KEN']
for dataset in dataset_list:
    df = pd.read_csv(f'./Datasets/{dataset}.csv', thousands=',')
    clean_investingcom_df(df, dataset, dataset.split('-')[1])

In [26]:
# Clean Macrotrends Datasets
# These datasets provide us the value of the index on each business day between certain dates
# We need to find the values on the first business day of each year, so we can calculate YoY change.

def clean_macrotrend_df(df, name, country_code):
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    df['Day_Change'] = df['value'].pct_change()*100
    volatility = df.groupby(by='year')['Day_Change'].std() * (df.groupby(by='year').size())**0.5

    # Get the first row representing the first business day of each year
    # Code From: https://stackoverflow.com/questions/71002941/get-the-first-row-of-each-group-of-unique-values-in-another-column 
    df = df.groupby('year', as_index=False).first()
    df['volatility'] = volatility.values

    df['YoY Change'] = df['value'].pct_change()*100
    # Shift YoY Change figures up by 1 row so that YoY Change over 1 year matches with that year for plotting later.
    df['YoY Change'] = df['YoY Change'].shift(-1)

    df.dropna(inplace=True)
    df.drop(columns=['month', 'day', 'date', 'value', 'Day_Change'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['code'] = country_code
    # Remove first row since it might not be a full year depending on dataset
    df = df.iloc[1:]
    df = df[['year', 'YoY Change', 'volatility', 'code']]

    df.to_csv(f'./Datasets/{name}-Clean.csv', index=False)

In [27]:
# Clean Macrotrends Datasets
# Run function on all of our current MacroTrends Datasets
# https://www.macrotrends.net/charts/stock-indexes
dataset_list = ['BOVESPA-BRA', 'CAC40-FRA', 'DAX30-DEU', 'HangSeng-HKG', 'Nikkei225-JPN']
for dataset in dataset_list:
    df = pd.read_csv(f'./Datasets/{dataset}.csv')
    clean_macrotrend_df(df, dataset, dataset.split('-')[1])


10.837159716646337
1.3604038469467685
1.3887348467970342
1.6169986807891346
1.2302446584311397


In [33]:
nifty = pd.read_csv('./Datasets/Nifty50-IND.csv')
nifty = nifty[['Date', 'Close']]
nifty['date'] = pd.to_datetime(nifty['Date'])
nifty['year'] = nifty['date'].dt.year
nifty['month'] = nifty['date'].dt.month
nifty['day'] = nifty['date'].dt.day

nifty['Day_Change'] = nifty['Close'].pct_change()*100
volatility = nifty.groupby(by='year')['Day_Change'].std() * (nifty.groupby(by='year').size())**0.5

# Get the first row representing the first business day of each year
nifty = nifty.groupby('year', as_index=False).first()

nifty['volatility'] = volatility.values

nifty = nifty.iloc[1:]


nifty['YoY Change'] = nifty['Close'].pct_change()*100
nifty['YoY Change'] = nifty['YoY Change'].shift(-1)
nifty['code'] = 'IND'
nifty.dropna(inplace=True)
nifty.drop(columns=['month', 'day', 'date', 'Date', 'Close', 'Day_Change'], inplace=True)
nifty.to_csv('./Datasets/Nifty50-IND-Clean.csv', index=False)

In [6]:
import pandas as pd
import numpy as np
import altair as alt
KOF=pd.read_csv('Datasets/KOFGI_2022_public_original.csv')

KOF=KOF.dropna() # droping missing value 
KOF_df=pd.DataFrame(KOF)
KOF_df.head(10) # Overview of the  KOF data set


,code,country,year,KOFGI,KOFGIdf,KOFGIdj,KOFEcGI,KOFEcGIdf,KOFEcGIdj,KOFTrGI,...,KOFIpGIdj,KOFInGI,KOFInGIdf,KOFInGIdj,KOFCuGI,KOFCuGIdf,KOFCuGIdj,KOFPoGI,KOFPoGIdf,KOFPoGIdj
158,AGO,Angola,1975,27.0,33.0,21.0,33.0,38.0,28.0,40.0,...,11.0,25.0,39.0,11.0,19.0,29.0,10.0,29.0,35.0,23.0
159,AGO,Angola,1976,30.0,33.0,26.0,33.0,38.0,28.0,40.0,...,14.0,25.0,39.0,11.0,20.0,29.0,10.0,37.0,35.0,38.0
160,AGO,Angola,1977,30.0,33.0,27.0,33.0,38.0,28.0,40.0,...,11.0,25.0,39.0,11.0,20.0,29.0,11.0,39.0,35.0,42.0
161,AGO,Angola,1978,30.0,33.0,28.0,33.0,38.0,28.0,40.0,...,12.0,25.0,39.0,11.0,18.0,29.0,8.0,40.0,35.0,45.0
162,AGO,Angola,1979,28.0,33.0,22.0,33.0,37.0,28.0,40.0,...,12.0,25.0,39.0,11.0,18.0,29.0,8.0,32.0,35.0,29.0
163,AGO,Angola,1980,30.0,33.0,27.0,33.0,37.0,28.0,39.0,...,12.0,25.0,40.0,11.0,18.0,28.0,8.0,39.0,35.0,44.0
164,AGO,Angola,1981,28.0,34.0,22.0,34.0,39.0,28.0,41.0,...,12.0,25.0,38.0,11.0,18.0,27.0,9.0,31.0,35.0,26.0
165,AGO,Angola,1982,29.0,32.0,27.0,32.0,36.0,28.0,38.0,...,13.0,24.0,37.0,11.0,18.0,26.0,10.0,37.0,35.0,40.0
166,AGO,Angola,1983,30.0,32.0,28.0,32.0,36.0,28.0,39.0,...,13.0,23.0,36.0,11.0,17.0,26.0,8.0,40.0,35.0,45.0
167,AGO,Angola,1984,31.0,32.0,30.0,34.0,37.0,31.0,40.0,...,13.0,23.0,34.0,11.0,17.0,25.0,9.0,42.0,35.0,49.0


We will only work with the following countries for the period from 1990 until 2020:
United States of America (USA), Japan (JPN), France (FRA), Germany (DEU), Brasil (BRA), China (CHN), India (IND), Hong Kong (HKG), South Africa (ZAF), Marocco (MAR) and Kenya (KEN)

In [7]:
# let's select the countries that interest us
KOF_df = KOF_df[(KOF_df['code'] == 'USA') | (KOF_df['code'] == 'JPN') | (KOF_df['code'] == 'FRA') | 
    (KOF_df['code'] == 'DEU') | (KOF_df['code'] == 'BRA') | (KOF_df['code'] == 'CHN') | 
    (KOF_df['code'] == 'IND') | (KOF_df['code'] == 'HKG') | (KOF_df['code'] == 'KEN') |
    (KOF_df['code'] == 'MAR') | (KOF_df['code'] == 'ZAF')]

# Removed South africa because of missing dataset
# | (KOF_df['code'] == 'ZAF')
#  Now let's select years between 1990 and 2020
KOF_df=KOF_df[(KOF_df['year'] >= 1990) & (KOF_df['year'] <= 2020)]

# Here is a overview of our new data frame
KOF_df.head(10)
KOF_df.to_csv('Datasets/KOF_clean.csv', index=False)

In [4]:
# Script that cleans the world bank index data: https://data.worldbank.org/indicator/CM.MKT.INDX.ZG?view=map
# and merges it with the KOF data

wb_df = pd.read_csv('./Datasets/World_Bank_Indices.csv')
wb_df = wb_df.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 67'], axis=1)

excluded_years = [str(i) for i in range(1960, 1995)]
active_years = [str(i) for i in range(1995, 2022)]

# Drop years that are not in our active time period
wb_df = wb_df.drop(excluded_years, axis=1)

# Keep only countries with data for at least one year in our active time period
wb_df = wb_df.iloc[wb_df[active_years].dropna(axis=0, how='all').index]

# Transform into long format, with a row for each year
wb_df = pd.melt(wb_df, id_vars=['Country Name', 'Country Code'], value_vars=active_years, var_name='Year', value_name='Pct Change').dropna()

#Prep for merge with kof df by making column names match
wb_df['year'] = wb_df['Year'].astype(int)
wb_df['code'] = wb_df['Country Code']
wb_df['country'] = wb_df['Country Name']
wb_df = wb_df.drop(['Country Code', 'Year', 'Country Name'], axis=1)

kof_df = pd.read_csv('./Datasets/KOFGI_2022_public_original.csv')
# Merge dataframes and drop the extra country name column
merged_df = pd.merge(wb_df, kof_df, how='inner', left_on=['code', 'year'], right_on=['code', 'year'], suffixes=('', '_duplicated'))

merged_df.to_csv('./Datasets/KOF_World_Bank_merged.csv', index=False)